In [29]:
import os
import zipfile
from dataclasses import dataclass
from pathlib import Path
from urllib import request

In [30]:
# !ls

In [31]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_path: Path
    unzip_dir: Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: Path
    ALL_REQUIRED_FILES: list

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [32]:
from src.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.utils import read_yaml, get_size
from src.logger import logging
from src.exception import CustomException

In [33]:
h=read_yaml(Path(f'../{CONFIG_FILE_PATH}'))
h.data_ingestion.root_dir

['artifacts', 'data_ingestion.ipynb', 'logs', 'text_summerization.ipynb']
[2023-07-18 22:49:07,524] 23 root - INFO - Yaml Config file successfully loaded from path: ..\config\config.yaml


'artifacts/data_ingestion'

In [34]:
class ConfigurationManager:
    def __init__(self, config_file_path=Path(f'../{CONFIG_FILE_PATH}'),
    param_file_path=Path(f'../{PARAMS_FILE_PATH}')) -> None:
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(param_file_path)

        os.makedirs(self.config.artifacts_root, exist_ok=True)
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        os.makedirs(config.root_dir, exist_ok=True)

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_path=config.local_data_path,
            unzip_dir=config.unzip_dir 
        )
        return data_ingestion_config
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        os.makedirs(config.root_dir, exist_ok=True)

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES 
        )
        return data_validation_config
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        os.makedirs(config.root_dir, exist_ok=True)

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config
    
    def get_model_trainer_config(self):
        config = self.config.model_trainer
        os.makedirs(config.root_dir, exist_ok=True)

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt
        )
        return model_trainer_config
    
    def get_model_evaluation_config(self):
        config = self.config.model_evaluation
        os.makedirs(config.root_dir, exist_ok=True)

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt
        )
        return model_evaluation_config

In [35]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_dataset(self):
        if not os.path.exists(self.config.local_data_path):
            file, header = request.urlretrieve(url=self.config.source_URL,
                                               filename=self.config.local_data_path)
            logging.info(f"{file} successfully downloaded! Info: \n{header}")
        else:
            logging.info(f"File already exists of size: {get_size(Path(self.config.local_data_path))} KB")

    def extract_zip_file(self, unzip_path=None):
        """Extract zip file to specified path

        Args:
            unzip_path: str, optional
                The path to extract the zip file to. If not provided, it defaults to `self.config.unzip_dir`.
        """
        if unzip_path is None:
            unzip_path = self.config.unzip_dir
        try:
            os.makedirs(unzip_path, exist_ok=True)
            with zipfile.ZipFile(self.config.local_data_path, 'r') as zip_fp:
                zip_fp.extractall(unzip_path)
        except CustomException as err:
            logging.warn(f"Error while extracting file path: {unzip_path}")
            logging.error(f"Error explaination: {err}")


## Data Validation

In [36]:
class DataValidation:
    def __init__(self, config: DataValidationConfig) -> None:
        self.config = config
    
    def validate_files(self):
        try:
            validation_status = False
            all_files = os.listdir(os.path.join("artifacts", "data_ingestion", "samsum_dataset"))
            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as fp:
                        fp.write(f"Validation status of {file}: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as fp:
                        fp.write(f"Validation status of {file}: {validation_status}")
                logging.info(f"{file} Files validation Success. VALIDATION_STATUS: {validation_status}")
            return validation_status
        
        except Exception as err:
            logging.warn(f"Cannot validate due to error: \n{err}")
            raise err

In [37]:
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [38]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name, use_fast=False)
    
    def convert_to_features(self, batch_data):
        # Tokenize the dialogue in the batch using the tokenizer
        input_encodings = self.tokenizer(batch_data['dialogue'], max_length=1024, truncation=True)

        # Tokenize the summary in the batch using the tokenizer as the target tokenizer
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(batch_data['summary'], max_length=128, truncation=True)

        # Return the converted features as a dictionary
        return {
            'input_ids': input_encodings['input_ids'],         # Input token IDs
            'attention_mask': input_encodings['attention_mask'],  # Attention mask
            'labels': target_encodings['input_ids']             # Target token IDs for the model's training
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

import pandas as pd

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)


        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
  
        # rouge_metric = load_metric('rouge')

        # score = self.calculate_metric_on_test_ds(
        # dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
        #     )

        rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

        df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_file_name, index=False)

In [41]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_dataset()
    data_ingestion.extract_zip_file()
    
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_files()

    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()

    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()

    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    # model_evaluation_config.evaluate()
except Exception as e:
    raise e

['artifacts', 'data_ingestion.ipynb', 'logs', 'text_summerization.ipynb']
[2023-07-18 23:06:05,227] 23 root - INFO - Yaml Config file successfully loaded from path: ..\config\config.yaml


['artifacts', 'data_ingestion.ipynb', 'logs', 'text_summerization.ipynb']
[2023-07-18 23:06:05,408] 23 root - INFO - Yaml Config file successfully loaded from path: ..\params.yaml
[2023-07-18 23:06:05,491] 11 root - INFO - File already exists of size: 7718.35 KB
[2023-07-18 23:06:08,166] 18 root - INFO - dataset_dict.json Files validation Success. VALIDATION_STATUS: False
[2023-07-18 23:06:08,196] 18 root - INFO - test Files validation Success. VALIDATION_STATUS: True
[2023-07-18 23:06:08,220] 18 root - INFO - train Files validation Success. VALIDATION_STATUS: True
[2023-07-18 23:06:08,240] 18 root - INFO - validation Files validation Success. VALIDATION_STATUS: True
